In [3]:
import nltk
from nltk.corpus import wordnet
import io
import sys
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import math

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    
def word_probabilities(dic, sub_total):#우도 계산
    for i in range(0, 4):
        for w in dic[i]:
            dic[i][w]=(word_list[i][w])/(sub_total[i])

def get_log_value(test_data, dic, weight):#값 추출
    ans=-2100000000
    ans_result=0
    for i in range(0, 4):
        result=0
        for w in test_data:
            if w in dic[i]:
               # print(w+" "+str(dic[i][w])+" "+str(test_data[w]))
                result+=(math.log(dic[i][w], 10)*test_data[w])
            else:
                result+=math.log(weight, 10)
        if (ans<result):
            ans=result
            ans_result=i
    return ans_result

nltk.download("stopwords")#불용어 사전 다운로드
nltk.download('averaged_perceptron_tagger')#이건 왜 다운받았더라..?
stop_words=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()

average=0
ans_cnt=0
cnt=0

for k in range(1, 26):
    word_list=[]#word_list[0]= interest 단어 딕셔너리, word_list[1]=jobs 단어 딕셔너리, word_list[2]= money_supply 단어 딕셔너리, word_list[3]=trade 단어 딕셔너리
    cartegory=["interest", "jobs", "money_supply", "trade"]
    total=0
    sub_total=[]# 각 카테고리 별 단어의 횟수 총합
    dic=[]#확률 계산을 담는 리스트형 딕셔너리
    
    for i in range(0, 4):
        words={}
        for j in range(1, 26):
            if j==k:
                continue
            f="dataset\\dev\\"+cartegory[i]+"\\"+cartegory[i]+"_"+str(j)+".txt"
            sentence=""
            word_tokens=[]#tokenize를 담는 리스트
            li=[]#불용어를 제외한 리스트
            lemmatize=[]#표제어
            compare=[]#표제어 한 단어들의 품사 태깅 정보
        
            with open(f, 'r') as train:
                sentence=train.read()#파일 읽기
                sentence=sentence.lower()#소문자로 치환
                #특수문자, 숫자 제거
                parse=re.sub("""[=.#/?:$}()&;*^%@!+<>,"']|[0-9]""", "", sentence)
                #탭, 엔터 제거
                parse=re.sub("[\s]|[-]", " ", parse)
                #공백 여러개 하나로 치환
                parse=re.sub(" +", " ", parse)
                #토큰 분리
                word_tokens=word_tokenize(parse)
        
            #불용어 제거
            for w in word_tokens:
                if w not in stop_words:
                    li.append(w)
        
            #품사태깅
            pos_li=nltk.pos_tag(li)
        
            #표제어화
            for w in pos_li:
                try:
                    tag=get_wordnet_pos(w[1])
                    lemmatize.append(lemmatizer.lemmatize(w[0], tag))
                except Exception as e:
                    lemmatize.append("TagExceptionError")#tag가 none인 경우

            compare=nltk.pos_tag(lemmatize)
        
            #품사 태깅을 통한 lemmatize
            for (p, c) in zip (pos_li, compare):
                if c[0]!="TagExceptionError":#tag가 none이 아닌 경우
                    tag1=get_wordnet_pos(p[1])
                    tag2=get_wordnet_pos(c[1])
                    if(tag1==tag2):#둘의 tag가 같은 경우
                        if(c[0] in words):
                            words[c[0]]+=1
                        else:
                            words[c[0]]=1
                    elif(tag1=='v' and tag2=='n'):#표제어화 했을때 동사에서 명사로 바뀐경우
                        if(c[0] in words):
                            words[c[0]]+=1
                        else:
                            words[c[0]]=1
                    elif(tag1=='v' and tag2=='r'):#표제어화 했을때 동사에서 형용사로 바뀐경우
                        if(c[0] in words):
                            words[c[0]]+=1
                        else:
                            words[c[0]]=1
                    else:#둘의 tag가 다른경우 (ex> goods->good)
                        if(p[0] in words):
                            words[p[0]]+=1
                        else:
                            words[p[0]]=1
                    
        word_list.append(words)
    
    for i in range(0, 4):
        tmp=0
        sub_dic={}
        for w in word_list[i]:
            tmp+=word_list[i][w]
            sub_dic[w]=0
        sub_total.append(tmp)
        dic.append(sub_dic)
    
    word_probabilities(dic, sub_total)
    
    test_data=[]
    for i in range(0, 4):
        test_words={}
        test_file="dataset\\dev\\"+cartegory[i]+"\\"+cartegory[i]+"_"+str(k)+".txt"
        test_sentence=""
        test_word_tokens=[]
        test_li=[]
        test_lemmatize=[]
        test_compare=[]
        
        with open(test_file, 'r') as test:
            test_sentence=test.read()
            test_sentence=test_sentence.lower()
            test_parse=re.sub("""[=.#/?:$}()&;*^%@!+<>,"']|[0-9]""", "", test_sentence)
            test_parse=re.sub("[\s]|[-]", " ", test_parse)
            #공백 여러개 하나로 치환
            test_parse=re.sub(" +", " ", test_parse)
            #토큰 분리
            test_word_tokens=word_tokenize(test_parse)
        
        for w in test_word_tokens:
            if w not in stop_words:
                test_li.append(w)
        
        test_pos_li=nltk.pos_tag(test_li)
        
        for w in test_pos_li:
            try:
                tag=get_wordnet_pos(w[1])
                test_lemmatize.append(lemmatizer.lemmatize(w[0], tag))
            except Exception as e:
                test_lemmatize.append("TagExceptionError")#tag가 none인 경우

        test_compare=nltk.pos_tag(test_lemmatize)
        
        for (p, c) in zip (test_pos_li, test_compare):
            if c[0]!="TagExceptionError":#tag가 none이 아닌 경우
                tag1=get_wordnet_pos(p[1])
                tag2=get_wordnet_pos(c[1])
                if(tag1==tag2):#둘의 tag가 같은 경우
                    if(c[0] in test_words):
                        test_words[c[0]]+=1
                    else:
                        test_words[c[0]]=1
                elif(tag1=='v' and tag2=='n'):#표제어화 했을때 동사에서 명사로 바뀐경우
                    if(c[0] in test_words):
                        test_words[c[0]]+=1
                    else:
                        test_words[c[0]]=1
                elif(tag1=='v' and tag2=='r'):#표제어화 했을때 동사에서 형용사로 바뀐경우
                    if(c[0] in test_words):
                        test_words[c[0]]+=1
                    else:
                        test_words[c[0]]=1
                else:#둘의 tag가 다른경우 (ex> goods->good)
                    if(p[0] in test_words):
                        test_words[p[0]]+=1
                    else:
                        test_words[p[0]]=1
                
        test_data.append(test_words)
    
    for z in range(0, 4):
        cnt+=1
        tmp=get_log_value(test_data[z], dic, 0.0000000005)
        if(tmp==z):
            ans_cnt+=1
        print(cartegory[z]+"_"+str(k)+".txt의 문서 분류 ="+cartegory[tmp]+" 정확도 "+str((ans_cnt/cnt)*100)+"%")
        

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhdqu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dhdqu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


interest_1.txt의 문서 분류 =trade 정확도 0.0%
jobs_1.txt의 문서 분류 =jobs 정확도 50.0%
money_supply_1.txt의 문서 분류 =trade 정확도 33.33333333333333%
trade_1.txt의 문서 분류 =trade 정확도 50.0%
interest_2.txt의 문서 분류 =interest 정확도 60.0%
jobs_2.txt의 문서 분류 =jobs 정확도 66.66666666666666%
money_supply_2.txt의 문서 분류 =money_supply 정확도 71.42857142857143%
trade_2.txt의 문서 분류 =trade 정확도 75.0%
interest_3.txt의 문서 분류 =interest 정확도 77.77777777777779%
jobs_3.txt의 문서 분류 =jobs 정확도 80.0%
money_supply_3.txt의 문서 분류 =money_supply 정확도 81.81818181818183%
trade_3.txt의 문서 분류 =trade 정확도 83.33333333333334%
interest_4.txt의 문서 분류 =interest 정확도 84.61538461538461%
jobs_4.txt의 문서 분류 =jobs 정확도 85.71428571428571%
money_supply_4.txt의 문서 분류 =money_supply 정확도 86.66666666666667%
trade_4.txt의 문서 분류 =trade 정확도 87.5%
interest_5.txt의 문서 분류 =trade 정확도 82.35294117647058%
jobs_5.txt의 문서 분류 =jobs 정확도 83.33333333333334%
money_supply_5.txt의 문서 분류 =money_supply 정확도 84.21052631578947%
trade_5.txt의 문서 분류 =trade 정확도 85.0%
interest_6.txt의 문서 분류 =interest 정확도 85.714285714

In [4]:
word_list=[]#word_list[0]= interest 단어 딕셔너리, word_list[1]=jobs 단어 딕셔너리, word_list[2]= money_supply 단어 딕셔너리, word_list[3]=trade 단어 딕셔너리
cartegory=["interest", "jobs", "money_supply", "trade"]
total=0
sub_total=[]# 각 카테고리 별 단어의 횟수 총합
dic=[]#확률 계산을 담는 리스트형 딕셔너리



for i in range(0, 4):# 각 분야별로 전처리
    words={}
    for j in range(1, 26):
        f="dataset\\dev\\"+cartegory[i]+"\\"+cartegory[i]+"_"+str(j)+".txt"
        sentence=""
        word_tokens=[]#tokenize를 담는 리스트
        li=[]#불용어를 제외한 리스트
        lemmatize=[]#표제어
        compare=[]#표제어 한 단어들의 품사 태깅 정보
        
        with open(f, 'r') as train:
            sentence=train.read()#파일 읽기
            sentence=sentence.lower()#소문자로 치환
            #특수문자, 숫자 제거
            parse=re.sub("""[=.#/?:$}()&;*^%@!+<>,"']|[0-9]""", "", sentence)
            #탭, 엔터 제거
            parse=re.sub("[\s]|[-]", " ", parse)
            #공백 여러개 하나로 치환
            parse=re.sub(" +", " ", parse)
            #토큰 분리
            word_tokens=word_tokenize(parse)
        
        #불용어 제거
        for w in word_tokens:
            if w not in stop_words:
                li.append(w)
        
        #품사태깅
        pos_li=nltk.pos_tag(li)
        
        #표제어화
        for w in pos_li:
            try:
                tag=get_wordnet_pos(w[1])
                lemmatize.append(lemmatizer.lemmatize(w[0], tag))
            except Exception as e:
                lemmatize.append("TagExceptionError")#tag가 none인 경우

        compare=nltk.pos_tag(lemmatize)
        
        #품사 태깅을 통한 lemmatize
        for (p, c) in zip (pos_li, compare):
            if c[0]!="TagExceptionError":#tag가 none이 아닌 경우
                tag1=get_wordnet_pos(p[1])
                tag2=get_wordnet_pos(c[1])
                if(tag1==tag2):#둘의 tag가 같은 경우
                    if(c[0] in words):
                        words[c[0]]+=1
                    else:
                        words[c[0]]=1
                elif(tag1=='v' and tag2=='n'):#표제어화 했을때 동사에서 명사로 바뀐경우
                    if(c[0] in words):
                        words[c[0]]+=1
                    else:
                        words[c[0]]=1
                elif(tag1=='v' and tag2=='r'):#표제어화 했을때 동사에서 형용사로 바뀐경우
                    if(c[0] in words):
                        words[c[0]]+=1
                    else:
                        words[c[0]]=1
                else:#둘의 tag가 다른경우 (ex> goods->good)
                    if(p[0] in words):
                        words[p[0]]+=1
                    else:
                        words[p[0]]=1

    word_list.append(words)
    
for i in range(0, 4):
    tmp=0
    sub_dic={}
    for w in word_list[i]:
        tmp+=word_list[i][w]
        sub_dic[w]=0
    sub_total.append(tmp)
    dic.append(sub_dic)

word_probabilities(dic, sub_total)
test_data=[]

for i in range(1, 21):
    words={}
    f="dataset\\test\\"+str(i)+".txt"
    sentence=""
    word_tokens=[]#tokenize를 담는 리스트
    li=[]#불용어를 제외한 리스트
    lemmatize=[]#표제어
    compare=[]#표제어 한 단어들의 품사 태깅 정보
        
    with open(f, 'r') as train:
        sentence=train.read()#파일 읽기
        sentence=sentence.lower()#소문자로 치환
        #특수문자, 숫자 제거
        parse=re.sub("""[=.#/?:$}()&;*^%@!+<>,"']|[0-9]""", "", sentence)
        #탭, 엔터 제거
        parse=re.sub("[\s]|[-]", " ", parse)
        #공백 여러개 하나로 치환
        parse=re.sub(" +", " ", parse)
        #토큰 분리
        word_tokens=word_tokenize(parse)
    #불용어 제거
    for w in word_tokens:
        if w not in stop_words:
            li.append(w)
        
    #품사태깅
    pos_li=nltk.pos_tag(li)
        
    #표제어화
    for w in pos_li:
        try:
            tag=get_wordnet_pos(w[1])
            lemmatize.append(lemmatizer.lemmatize(w[0], tag))
        except Exception as e:
            lemmatize.append("TagExceptionError")#tag가 none인 경우

    compare=nltk.pos_tag(lemmatize)
        
    #품사 태깅을 통한 lemmatize
    for (p, c) in zip (pos_li, compare):
        if c[0]!="TagExceptionError":#tag가 none이 아닌 경우
            tag1=get_wordnet_pos(p[1])
            tag2=get_wordnet_pos(c[1])
            if(tag1==tag2):#둘의 tag가 같은 경우
                if(c[0] in words):
                    words[c[0]]+=1
                else:
                    words[c[0]]=1
            elif(tag1=='v' and tag2=='n'):#표제어화 했을때 동사에서 명사로 바뀐경우
                if(c[0] in words):
                    words[c[0]]+=1
                else:
                    words[c[0]]=1
            elif(tag1=='v' and tag2=='r'):#표제어화 했을때 동사에서 형용사로 바뀐경우
                if(c[0] in words):
                    words[c[0]]+=1
                else:
                    words[c[0]]=1
            else:#둘의 tag가 다른경우 (ex> goods->good)
                if(p[0] in words):
                    words[p[0]]+=1
                else:
                    words[p[0]]=1
    test_data.append(words)

for z in range(1, 21):
    print(str(z)+"번 파일 "+cartegory[get_log_value(test_data[z-1], dic, 0.0000000005)])

1번 파일 interest
2번 파일 money_supply
3번 파일 interest
4번 파일 jobs
5번 파일 money_supply
6번 파일 jobs
7번 파일 trade
8번 파일 jobs
9번 파일 jobs
10번 파일 interest
11번 파일 money_supply
12번 파일 trade
13번 파일 money_supply
14번 파일 trade
15번 파일 money_supply
16번 파일 trade
17번 파일 interest
18번 파일 trade
19번 파일 interest
20번 파일 trade
